# DAY 1

In [11]:
input_list = open("./puzzle_inputs/day1.txt").read().split("\n")

In [12]:
# Only for digits:
sum = 0

for row in input_list:
    row_digits = [char for char in row if char.isdigit()]
    calibration_value = int(row_digits[0]) * 10 + int(row_digits[-1])
    sum += calibration_value
print(sum)

55712


In [14]:
# Now with the numners spelled out:
import re

words2digits = {
    "one": "1",
    "two": "2",
    "three": "3",
    "four": "4",
    "five": "5",
    "six": "6",
    "seven": "7",
    "eight": "8",
    "nine": "9",
    "zero": "0",
}

sum = 0
valid_digit_regex = r"(?=(\d|one|two|three|four|five|six|seven|eight|nine|zero))"
for row in input_list:
    row_digits = re.findall(valid_digit_regex, row)
    for i, rd in enumerate(row_digits):
        if not rd.isdigit():
            row_digits[i] = words2digits[rd]

    calibration_value = int(row_digits[0] + row_digits[-1])
    sum += calibration_value
print(sum)

55413


---
# DAY 2

In [170]:
import urllib
import re
from dataclasses import dataclass

# read data from the downloaded txt
puzzle_input = open("./puzzle_inputs/day2.txt").read().split("\n")

In [228]:
# Some basic order in the data that we are getting
@dataclass
class cubeDraw:
    red: int = 0
    blue: int = 0
    green: int = 0


@dataclass
class drawList:
    ID: int
    drawList: list[cubeDraw]
    minimunSet: cubeDraw  # The minimun set has the same structure as a draw, used in part 2


# Regex for finding the values of red, blue and green
rregex = r"(\d+) red"
bregex = r"(\d+) blue"
gregex = r"(\d+) green"

game_draws = []

for game in puzzle_input:
    gameID = game.split(":")[0].split(" ")[1]
    drawlist = game.split(":")[1:][0].split(";")

    numeric_draws = []
    for d in drawlist:
        draw = cubeDraw()
        if "red" in d:
            draw.red = int(re.search(rregex, d).group(1))
        if "blue" in d:
            draw.blue = int(re.search(bregex, d).group(1))
        if "green" in d:
            draw.green = int(re.search(gregex, d).group(1))
        numeric_draws.append(draw)
    game_draws.append(drawList(gameID, numeric_draws, cubeDraw()))

In [230]:
# Now we iterate over all draws in game_draws and find those who have a invalid number of cubes in any draw:
# Stage 1: cube limits

# max_cubes = {"red": 12, "green": 13, "blue": 14}

# A game is valid if all of it's draws have a number of cubes of each color less than or equal to the maximun number of cubes for each color.
invalid_game_IDS = []
for game in game_draws:
    for draw in game.drawList:
        if (draw.red > 12) | (draw.green > 13) | (draw.blue > 14):
            invalid_game_IDS.append(int(game.ID))
            break

# Using sets isntead of logic beacuse I'm a moron
all_games_id = set([int(game.ID) for game in game_draws])
invalid_games_id = set(invalid_game_IDS)

valid_games = all_games_id.difference(invalid_game_IDS)
sum = 0
for id in valid_games:
    sum += id
print(sum)

2348


# Part 2. Porwesets

In [237]:
# Find the power of the minimun set of cubes in each game, then add them together
""" 
    I retroactivelly added the minimunSet attribute into the drawlist class
"""

# For each game, find the maximun number of cubes that must have been present (max red, green and blue for all draws in a game)
game_minset = {"red": 0, "green": 0, "blue": 0}

for game in game_draws:
    game.minimunSet = cubeDraw(
        max([draw.red for draw in game.drawList]),
        max([draw.green for draw in game.drawList]),
        max([draw.blue for draw in game.drawList]),
    )

In [238]:
# Calculate the power for all minimun set in every game
sum_of_powers = 0
for game in game_draws:
    sum_of_powers += game.minimunSet.red * game.minimunSet.green * game.minimunSet.blue
print(sum_of_powers)

76008


---
# Day 3

In [251]:
# Load the puzzle data

puzzle_input = open("./puzzle_inputs/day3.txt").read().split("\n")

In [272]:
# Lets load it into a matrix of shape 140x140, the diagonal and vertical symbols are important so we need to preserve the structure.
import numpy as np

schematic = np.chararray((len(puzzle_input), len(puzzle_input[0])))

In [274]:
for i, charlist in enumerate(puzzle_input):
    for j, char in enumerate(charlist):
        schematic[i, j] = char

In [277]:
schematic

chararray([[b'.', b'.', b'.', ..., b'5', b'4', b'1'],
           [b'.', b'.', b'.', ..., b'.', b'.', b'.'],
           [b'.', b'.', b'.', ..., b'4', b'6', b'6'],
           ...,
           [b'.', b'.', b'.', ..., b'.', b'.', b'.'],
           [b'.', b'.', b'.', ..., b'.', b'.', b'.'],
           [b'.', b'.', b'5', ..., b'.', b'.', b'.']], dtype='|S1')

In [392]:
# Step 1:
"""  
    Find all single digits that are next to a symbol
"""


def neighbourhood(i, j, arr):
    """
    Recover the nearest 3-8 elements of an array given a position (i,j)
    """
    top = max(i - 1, 0)
    left = max(j - 1, 0)
    bottom = min(i + 2, arr.shape[0])
    right = min(j + 2, arr.shape[1])
    result = arr[top:bottom, left:right]
    return result


def sybolFinder(i, j, charmatrix):
    """
    Given a position in the matrix, chech if the character in the 8 possible possitions is not "." or another number

    Carefull with the borders!
    """

    number_grid = neighbourhood(i, j, charmatrix)

    # Chech if the idex is adjacent to something other than a "." or another number.
    is_adjacent = any(
        [
            (
                ("." != str(n, encoding="ascii"))
                & (not bool(re.search(r"\d", str(n, encoding="ascii"))))
            )
            for n in number_grid.flatten()
        ]
    )

    return is_adjacent


def isNumber(i, j, charmatrix):
    return str(charmatrix[i, j], encoding="ascii").isdigit()

In [453]:
# Create the boolean array with the valid cells (those that contain a number and are adjacent to a symbol)

is_adjacent_matrix = np.empty(shape=schematic.shape, dtype=bool)
is_number_matrix = np.empty(shape=schematic.shape, dtype=bool)

# Find where there is a single digit and the cell is adjacent to a digit
is_target_matrix = np.empty(shape=schematic.shape, dtype=bool)

for i in range(schematic.shape[0]):
    for j in range(schematic.shape[1]):
        is_adjacent_matrix[i, j] = sybolFinder(i, j, schematic)
        is_number_matrix[i, j] = isNumber(i, j, schematic)
        is_target_matrix[i, j] = is_adjacent_matrix[i, j] & is_number_matrix[i, j]

In [445]:
# Now we recover all the digits in the same line as the target matrix shows, for this we can reconstitute the strings and work with a list of bools and char for each row of the matrix
matches = [x for x in re.finditer(pattern=r"\d+", string=puzzle_input[0])]

# chech if each span contains a True in the boolean list
valid_matches = []
for m in matches:
    if is_target_matrix[0, m.span()[0] : m.span()[1]].any():
        valid_matches.append(m.group())

In [449]:
# One wrapper more for all the rows in the puzzle input:
valid_matches = []
for i, row in enumerate(puzzle_input):
    matches = [m for m in re.finditer(pattern=r"\d+", string=row)]
    for m in matches:
        if is_target_matrix[i, m.span()[0] : m.span()[1]].any():
            valid_matches.append(m.group())

In [451]:
# adding all toguether:
sum = 0
for n in valid_matches:
    sum += int(n)
sum

530495

# Part 2

In [ ]:
# Finding gears and gear ratios
""" 
    We can reuse the neighbourhood functions tu find all elements that are * with two adjacent numbers
"""


# Finding their corresponding numbers

# Multiply and sum

In [454]:
# Finding the potential gears
is_asterisc_matrix = np.empty(shape=schematic.shape, dtype=bool)

for i in range(schematic.shape[0]):
    for j in range(schematic.shape[1]):
        is_asterisc_matrix[i, j] = str(schematic[i, j], encoding="ascii") == "*"

In [525]:
# For each asterisc, find the ones that are gears.
""" 
A gear is an asterisc that is touching 2 parts (distinct numbers).
Consecutive numbers should not be counted as two numbers.
"""

# Check if any row in the neighbourhood of an asterisc is [digit, "other", digit]. If it is -> gear

# elif two distict rows have any digits -> gear

# (else not a gear)


# Logic functions for finding the gears: ---
def rowlikeGear(nbh):
    """
    Returns if the neighbourhood of an asterisc have a [digit, other, digit] structure
    """

    for row in nbh:
        if (row[0].isdigit()) & (not row[1].isdigit()) & (row[2].isdigit()):
            return True

    return False


def columnlikeGear(nbh):
    """
    Returns if the neighbourhood of an asterisc presents a gear connecting two rows
    """

    votes = [any([char.isdigit() for char in row]) for row in nbh]
    votesum = 0
    for v in votes:
        votesum += v
    if votesum > 1:
        return True

    return False

In [546]:
# Inplementation
is_gear_matrix = np.empty(shape=schematic.shape, dtype=bool)
for i in range(schematic.shape[0]):
    for j in range(schematic.shape[1]):
        if is_asterisc_matrix[i, j] == True:
            nbh = neighbourhood(i, j, schematic)

            if rowlikeGear(nbh) == True:
                is_gear_matrix[i, j] = True

            elif columnlikeGear(nbh) == True:
                is_gear_matrix[i, j] = True

            else:
                is_gear_matrix[i, j] = False
        else:
            is_gear_matrix[i, j] = False

In [547]:
# Example
nbh = neighbourhood(40, 6, schematic)
nbh, rowlikeGear(nbh), columnlikeGear(nbh)

(chararray([[b'1', b'.', b'.'],
            [b'.', b'*', b'.'],
            [b'6', b'0', b'.']], dtype='|S1'),
 False,
 True)

In [1]:
# Now that we know the gears, lets find the numbers for each one.

--- 
# Day 4

In [18]:
#
from dataclasses import dataclass

In [3]:
# Load puzzle data
puzzle_input = open("./puzzle_inputs/day4.txt").read().split("\n")

In [16]:
row = puzzle_input[0]
row.split(":")[1].split("|")

[' 82 41 56 54 18 62 29 55 34 20 ',
 ' 37 14 10 80 58 11 65 96 90  8 59 32 53 21 98 83 17  9 87 25 71 77 70 73 24']

In [117]:
@dataclass
class Card:
    cardID: int
    winingNumbers: set[int]  # 10 winning numbers
    playedNumbers: set[int]  # 25 played numbers
    cardValue: int = 0
    copies: int = 1  # for part 2


# Excplicit definition of the class ?


# load the puzzle input into a list of Card objects:

card_list = []
for row in puzzle_input:
    cardID = int(row.split(":")[0].split(" ")[-1])

    playedNumbers = set(
        [
            int(char)
            for char in row.split(":")[1].split("|")[1].split(" ")
            if char.isdigit()
        ]
    )

    winingNumbers = set(
        [
            int(char)
            for char in row.split(":")[1].split("|")[0].split(" ")
            if char.isdigit()
        ]
    )

    card = Card(cardID=cardID, playedNumbers=playedNumbers, winingNumbers=winingNumbers)

    card_list.append(card)

# Example
inx = 42
print(puzzle_input[inx])
print(card_list[inx])

Card  43: 10 80 63 20 60 76 12 75 31  8 | 93 37 34 67 21 56 22 16 94 57 71 47 33 73 54 81 51 46 70 63 42 14 26 86 66
Card(cardID=43, winingNumbers={8, 10, 75, 76, 12, 31, 80, 20, 60, 63}, playedNumbers={14, 16, 21, 22, 26, 33, 34, 37, 42, 46, 47, 51, 54, 56, 57, 63, 66, 67, 70, 71, 73, 81, 86, 93, 94}, cardValue=0, copies=1)


In [118]:
# Calculate the value of the card:
"""  
    Chech how many numbers in winning numbers are in played numbers (N)
    cardValue = 2**(N-1)
"""

for card in card_list:
    common_digits = card.winingNumbers.intersection(card.playedNumbers)
    N = len(common_digits)
    if N > 0:
        card.cardValue = 2 ** (N - 1)

In [119]:
# Now we just add the number of points:
total = 0
for card in card_list:
    total += card.cardValue

print(total)

20667


---
# Part 2

We now add a new field to the dataclass (copies)

In [71]:
# @dataclass
# class Card:
#     cardID: int
#     winingNumbers: set[int]  # 10 winning numbers
#     playedNumbers: set[int]  # 25 played numbers
#     cardValue: int = 0  # We actually dont need this for the next part ...
#     copies: int = 1

In [115]:
# Now we process the last list, for card i:  the sqrt(card_value) = N is the number of cards with id i+1, i+2, ... , i+N that we need to increment the copies of.
import numpy as np

for card in card_list:
    N = int(np.sqrt(card.cardValue))
    if N > 0:
        for i in range(
            min(card.cardID, len(card_list) - 1),
            min((card.cardID + N), len(card_list) - 1),
            1,
        ):  # Index starts at 0, so we dont need to add 1 to the starting ID, since it starts at 1
            card_list[i].copies = card_list[i].copies + card.copies
            # TODO: This assignment seems to not do anything ??

In [116]:
# count the total number of cards:
total_copies = 0
for card in card_list:
    total_copies += card.copies

print(total_copies)

101044127248461257999323012787236688917103228652412


In [111]:
card_list[len(card_list) - 1]

Card(cardID=211, winingNumbers={97, 70, 41, 78, 51, 61, 22, 55, 26, 29}, playedNumbers={1, 2, 3, 5, 10, 12, 13, 15, 16, 23, 24, 35, 38, 40, 43, 45, 48, 52, 53, 66, 69, 71, 74, 77, 84}, cardValue=0, copies=1)

___
# Day 5


In [22]:
import pandas as pd

filepath = "./puzzle_inputs/day5.txt"
puzzle_input = open(filepath).read().split("\n")

In [21]:
# Custom functions to reed tables from a txt and get the correct mapiing when necessary


def read_table(file, table_name):
    table = {
        "destination_range_start": [],
        "source_range_start": [],
        "range_length": [],
    }
    start_reading = False

    with open(file, "r") as f:
        for line in f:
            line = line.strip()
            if line == f"{table_name}:":
                start_reading = True
                continue
            elif not line:
                start_reading = False
                continue

            if start_reading:
                data = line.split()
                for i, key in enumerate(table.keys()):
                    table[key].append(int(data[i]))

    return table


# TODO: I need to frame this into a MAP, so the computations are doable in part 2


def mapSourceToDestination(df, source_num):
    """
    Given a table and a source number, calculate the destination number acording
    to the logic in the puzzle
    """

    # Default answer, if the mapping does not occur, the source is maped as the same destination number
    destination_num = source_num

    for row in df.index:
        dest_r_start = df.iloc[row].iloc[0]
        source_r_start = df.iloc[row].iloc[1]
        r_length = df.iloc[row].iloc[2]

        # If the source number is in a valid interval, calculate the destination number
        if ((source_r_start + r_length) > source_num) & (source_num > source_r_start):
            destination_num = dest_r_start + (source_num - source_r_start)

    return destination_num

In [23]:
seed_list = [int(n) for n in puzzle_input[0].split(":")[1].split(" ") if n.isdigit()]
seed_to_soil = pd.DataFrame.from_dict(read_table(filepath, "seed-to-soil map"))

soil_to_fertilizer = pd.DataFrame.from_dict(
    read_table(filepath, "soil-to-fertilizer map")
)

fertilizer_to_water = pd.DataFrame.from_dict(
    read_table(filepath, "fertilizer-to-water map")
)

water_to_light = pd.DataFrame.from_dict(read_table(filepath, "water-to-light map"))

light_to_temperature = pd.DataFrame.from_dict(
    read_table(filepath, "light-to-temperature map")
)

temperature_to_humidity = pd.DataFrame.from_dict(
    read_table(filepath, "temperature-to-humidity map")
)

humidity_to_location = pd.DataFrame.from_dict(
    read_table(filepath, "humidity-to-location map")
)

In [24]:
# Using map():

soil_list = map(lambda x: mapSourceToDestination(seed_to_soil, x), seed_list)

fertilizer_list = map(
    lambda x: mapSourceToDestination(soil_to_fertilizer, x), soil_list
)

water_list = map(
    lambda x: mapSourceToDestination(fertilizer_to_water, x), fertilizer_list
)

ligth_list = map(lambda x: mapSourceToDestination(water_to_light, x), water_list)

temperature_list = map(
    lambda x: mapSourceToDestination(light_to_temperature, x), ligth_list
)

humidity_list = map(
    lambda x: mapSourceToDestination(temperature_to_humidity, x), temperature_list
)

location_list = map(
    lambda x: mapSourceToDestination(humidity_to_location, x), humidity_list
)

In [25]:
seed_to_location = pd.DataFrame(
    {
        "seed": seed_list,
        "location": list(location_list),
    }
)
seed_to_location.sort_values("location")

,seed,location
9,111959634,322500873
11,119609663,330150902
5,139859832,350401071
18,864476811,439018996
14,2035874278,629695516
2,3319921504,796058796
1,99920667,1241215496
17,55841637,1679819321
13,56288233,1680265917
4,67832336,1691810020


# Test data

In [18]:
testfilepath = "./puzzle_inputs/day5test.txt"

puzzle_input = open(testfilepath).read().split("\n")

# Starting seeds list
seed_list = [int(n) for n in puzzle_input[0].split(":")[1].split(" ") if n.isdigit()]

# Tables
seed_to_soil = pd.DataFrame.from_dict(read_table(testfilepath, "seed-to-soil map"))

soil_to_fertilizer = pd.DataFrame.from_dict(
    read_table(testfilepath, "soil-to-fertilizer map")
)

fertilizer_to_water = pd.DataFrame.from_dict(
    read_table(testfilepath, "fertilizer-to-water map")
)

water_to_light = pd.DataFrame.from_dict(read_table(testfilepath, "water-to-light map"))

light_to_temperature = pd.DataFrame.from_dict(
    read_table(testfilepath, "light-to-temperature map")
)

temperature_to_humidity = pd.DataFrame.from_dict(
    read_table(testfilepath, "temperature-to-humidity map")
)

humidity_to_location = pd.DataFrame.from_dict(
    read_table(testfilepath, "humidity-to-location map")
)

In [19]:
# Using map():

soil_list = map(lambda x: mapSourceToDestination(seed_to_soil, x), seed_list)

fertilizer_list = map(
    lambda x: mapSourceToDestination(soil_to_fertilizer, x), soil_list
)

water_list = map(
    lambda x: mapSourceToDestination(fertilizer_to_water, x), fertilizer_list
)

ligth_list = map(lambda x: mapSourceToDestination(water_to_light, x), water_list)

temperature_list = map(
    lambda x: mapSourceToDestination(light_to_temperature, x), ligth_list
)

humidity_list = map(
    lambda x: mapSourceToDestination(temperature_to_humidity, x), temperature_list
)

location_list = map(
    lambda x: mapSourceToDestination(humidity_to_location, x), humidity_list
)

In [20]:
seed_to_location = pd.DataFrame({"seed": seed_list, "location": list(location_list)})
seed_to_location

,seed,location
0,79,82
1,14,86
2,55,86
3,13,35


# Part 2

seed list actually describes ranges of seeds, the original numbers pair up in (start, length) pairs.

For this I need to rethink the problem enirely using mappings

In [26]:
seed_ranges = []
for i in range(len(seed_list) // 2):
    seed_ranges.append((seed_list[i], seed_list[i + 1]))

In [27]:
for sr in [seed_ranges[1]]:
    seed_list2 = range(sr[0], sr[1], 1)

    soil_list = map(lambda x: mapSourceToDestination(seed_to_soil, x), seed_list2)

    fertilizer_list = map(
        lambda x: mapSourceToDestination(soil_to_fertilizer, x), soil_list
    )

    water_list = map(
        lambda x: mapSourceToDestination(fertilizer_to_water, x), fertilizer_list
    )

    ligth_list = map(lambda x: mapSourceToDestination(water_to_light, x), water_list)

    temperature_list = map(
        lambda x: mapSourceToDestination(light_to_temperature, x), ligth_list
    )

    humidity_list = map(
        lambda x: mapSourceToDestination(temperature_to_humidity, x), temperature_list
    )

    location_list = map(
        lambda x: mapSourceToDestination(humidity_to_location, x), humidity_list
    )

    seed_to_location = pd.DataFrame(
        {
            "seed": seed_list,
            "location": location_list,
        }
    )
seed_to_location

KeyboardInterrupt: 